<a href="https://colab.research.google.com/github/kunalvshah/agentic-ai-tutorials/blob/main/Agentic_AI_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai pandas numpy yfinance faiss-cpu chromadb pypdf pdfplumber python-dotenv langchain-community openai



In [ ]:
from getpass import getpass
from google.colab import files
import pdfplumber
import yfinance as yf
from langchain.llms import OpenAI
import os

In [ ]:
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [ ]:
llm = OpenAI(temperature=0.5)

print(llm("Explain what is intrinsic value in stocks"))

In [ ]:
uploaded = files.upload()
text = ''
for filename in uploaded.keys():
    with pdfplumber.open(filename) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'

print(text[:2000])  # Show preview

In [ ]:
ticker = yf.Ticker("BRITANNIA.NS")
hist = ticker.history(period="5d")
print(hist)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = PyPDFLoader("britannia.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are a financial analyst. Use the following document to extract:

1. Total Revenue
2. Net Profit
3. EPS (Earnings Per Share)
4. Key management commentary
5. Any mention of business risks

If data is missing, respond 'Not Found'.

Document:
{context}

Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context"])

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_community.llms import OpenAI

llm = OpenAI(temperature=0.2)

qa_chain = LLMChain(llm=llm, prompt=prompt)

chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
result = chain.run(chunks)

print(result)

In [ ]:
# Let's assume it extracted EPS = 60, Sector PE = 25
EPS = 60
PE = 25
fair_price = EPS * PE
print(f"Estimated Fair Value: ₹{fair_price}")

In [ ]:
# Simple Reflex Agent in Python
def simple_agent():
    print("Welcome! Type 'hello', 'bye', or 'status'. Type 'exit' to stop.")
    while True:
        perception = input("You: ").strip().lower()
        if perception == "exit":
            print("Agent: Goodbye!")
            break
        elif perception == "hello":
            print("Agent: Hello there!")
        elif perception == "bye":
            print("Agent: See you soon!")
        elif perception == "status":
            print("Agent: I am a basic Python agent, running fine!")
        else:
            print("Agent: Sorry, I don't understand that command.")

simple_agent()


In [ ]:
class SimpleAgent:
    def __init__(self):
        self.memory = {}

    def perceive_and_act(self, perception):
        perception = perception.strip().lower()
        if perception == "hello":
            if not self.memory.get("greeted"):
                self.memory["greeted"] = True
                return "Agent: Hello! Nice to meet you."
            else:
                return "Agent: We've already greeted :)"
        elif perception == "bye":
            return "Agent: Goodbye!"
        elif perception == "status":
            times = self.memory.get("asked_status", 0) + 1
            self.memory["asked_status"] = times
            return f"Agent: You have asked for my status {times} time(s)."
        elif perception == "exit":
            return "exit"
        else:
            todo = self.memory.get("todo", [])
            todo.append(perception)
            self.memory["todo"] = todo
            return "To DO: " + ", ".join(todo)
            # return "Agent: I don't understand that command."

def run_agent():
    print("Type commands ('hello', 'bye', 'status', 'exit'):")
    agent = SimpleAgent()
    while True:
        user_input = input("You: ")
        response = agent.perceive_and_act(user_input)
        if response == "exit":
            print("Agent: Session ended.")
            break
        else:
            print(response)

run_agent()


In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain.llms import OpenAI
import os

from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

llm = OpenAI(temperature=0)  # adjust for more/less creativity

# Define tools (optionally add more, e.g., search, calculator)
tools = load_tools(["calculator"])

# Initialize agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Interact with agent (sample query)
agent.run("What is 2+2?")
